<a href="https://colab.research.google.com/github/AjayPatsariya/project-1-/blob/master/AI_Project_1_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tags = pd.read_csv('/content/drive/MyDrive/Tags.csv')

In [ ]:
tags.head()

,Id,Tag
0,80,flex
1,80,actionscript-3
2,80,air
3,90,svn
4,90,tortoisesvn


In [ ]:
tags.Tag.value_counts().nlargest(n=10)

javascript    124155
java          115212
c#            101186
php            98808
android        90659
jquery         78542
python         64601
html           58976
c++            47591
ios            47009
Name: Tag, dtype: int64

In [ ]:
tagtop10 = ['javascript','java','c#','php','android','jquery','python','html','c++','ios']

In [ ]:
tags_10 = tags[tags.Tag.isin(tagtop10)]
print(tags_10.shape)
tags_10.head()

(826739, 2)


,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [ ]:
tag10 = tags_10.loc[tags_10.astype(str).drop_duplicates().index]
tag10.shape

(826739, 2)

In [ ]:
tag10.head()

,Id,Tag
14,260,c#
18,330,c++
28,650,c#
35,930,c#
39,1010,c#


In [ ]:
def add_tags(question_id):
    return tags_10[tags_10['Id'] == question_id['Id']].Tag.values

top = tags_10.apply(add_tags, axis=1)

In [ ]:
len(top),tags_10.shape

(826739, (826739, 2))

In [ ]:
tag_top_10=pd.concat([tags_10, top.rename('Tags')], axis=1)
tag_top_10.head()

,Id,Tag,Tags
14,260,c#,[c#]
18,330,c++,[c++]
28,650,c#,[c#]
35,930,c#,[c#]
39,1010,c#,[c#]


In [ ]:
tag_top_10.drop(["Tag"], axis=1, inplace=True)
tag_top_10.shape

(826739, 2)

In [ ]:
top10_tags=tag_top_10.loc[tag_top_10.astype(str).drop_duplicates().index]

In [ ]:
top10_tags.shape

(706336, 2)

In [ ]:
ques = pd.read_csv('/content/drive/MyDrive/Questionfinal_clean.csv')

In [ ]:
ques.shape

(1264216, 3)

In [ ]:
data = pd.merge(ques, top10_tags, on = 'Id')
print(data.shape)
data.head()

(706336, 4)


,Id,Title,Body,Tags
0,260,Adding scripting functionality to .NET applica...,I have a little game written in C#. It uses a ...,[c#]
1,330,Should I use nested classes in this case?,I am working on a collection of classes used f...,[c++]
2,650,Automatically update version number,I would like the version property of my applic...,[c#]
3,930,How do I connect to a database and loop over a...,What's the simplest way to connect and query a...,[c#]
4,1010,"How to get the value of built, encoded ViewState?",I need to grab the base64-encoded representati...,[c#]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, BatchNormalization, GRU ,concatenate
from keras.models import Model
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
mlb = MultiLabelBinarizer()

In [ ]:
mlb.fit(data.Tags)
labels = mlb.classes_
labels

array(['android', 'c#', 'c++', 'html', 'ios', 'java', 'javascript',
       'jquery', 'php', 'python'], dtype=object)

In [ ]:
train,test = train_test_split(data[:550000],test_size=0.25,random_state=22)

In [ ]:
train.shape,test.shape

((412500, 4), (137500, 4))

In [ ]:
X_train_t = train['Title']
X_train_b = train['Body']
y_train = mlb.transform(train['Tags'])
X_test_t = test['Title']
X_test_b = test['Body']
y_test = mlb.transform(test['Tags'])

In [ ]:
sen_len_t = []
for sen in train['Title']:
    sen_len_t.append(len(word_tokenize(sen)))
max(sen_len_t)

60

In [ ]:
np.quantile(sen_len_t,0.97)

18.0

In [ ]:
max_len_t = 18
tok = Tokenizer(char_level=False,split=' ')
tok.fit_on_texts(X_train_t)
sequences_train_t = tok.texts_to_sequences(X_train_t)

In [ ]:
vocab_len_t = len(tok.index_word)
vocab_len_t

68975

In [ ]:
seq_mat_train_t = sequence.pad_sequences(sequences_train_t, maxlen = max_len_t)
seq_mat_train_t

array([[   0,    0,    0, ...,    2,  398,   89],
       [   0,    0,    0, ...,   21, 5663, 4317],
       [   0,    0,    0, ..., 5287,    2,   27],
       ...,
       [   0,    0,    0, ...,   30,    2,   18],
       [   0,    0,    0, ...,   49,    2,   18],
       [   0,    0,    0, ...,   26,  748,  199]], dtype=int32)

In [ ]:
sequences_test_t = tok.texts_to_sequences(X_test_t)
seq_mat_test_t = sequence.pad_sequences(sequences_test_t, maxlen = max_len_t)
seq_mat_test_t

array([[    0,     0,     0, ...,   132,     7,    78],
       [    0,     0,     0, ...,   236,   291,    68],
       [    0,     0,     0, ...,    25,  1263,   356],
       ...,
       [    0,     0,     0, ...,  2251,   236,   924],
       [    0,     0,     0, ..., 28520,  1638,   419],
       [    0,     0,     0, ...,    20, 56599,   630]], dtype=int32)

In [ ]:
seq_mat_train_t.shape, seq_mat_test_t.shape, y_train.shape, y_test.shape

((412500, 18), (137500, 18), (412500, 10), (137500, 10))

In [ ]:
sen_len_b = []

In [ ]:
for sen in train['Body']:
    sen_len_b.append(len(word_tokenize(sen)))
max(sen_len_b)

20868

In [ ]:
np.quantile(sen_len_b, 0.90)

578.0

In [ ]:
max_len_b = 578
tok.fit_on_texts(X_train_b)
sequence_train_b = tok.texts_to_sequences(X_train_b)

In [ ]:
vocab_len_b = len(tok.index_word)
vocab_len_b

1295441

In [ ]:
seq_mat_train_b = sequence.pad_sequences(sequence_train_b, maxlen = max_len_b)
seq_mat_train_b

array([[   0,    0,    0, ...,    3,  137,    2],
       [   0,    0,    0, ...,    1, 1579, 2041],
       [   0,    0,    0, ...,   85,   65,  427],
       ...,
       [   0,    0,    0, ...,  983,    2,  400],
       [   0,    0,    0, ...,  255,  336,   10],
       [   0,    0,    0, ...,  954,   24, 1427]], dtype=int32)

In [ ]:
sequence_test_b = tok.texts_to_sequences(X_test_b)
seq_mat_test_b = sequence.pad_sequences(sequence_test_b, maxlen = max_len_b)

In [ ]:
seq_mat_train_t.shape, seq_mat_train_b.shape, y_train.shape

((412500, 18), (412500, 578), (412500, 10))

In [ ]:
seq_mat_test_t.shape, seq_mat_test_b.shape, y_test.shape

((137500, 18), (137500, 578), (137500, 10))

In [ ]:
def RNN() :
    #Title i/p
    title_ip = Input(name='title_ip',shape=[max_len_t])
    title_ebb = Embedding(vocab_len_t+1, 100, input_length = max_len_t, mask_zero = True, name = 'title_ebb')(title_ip)
    GRU_out_t = GRU(40)(title_ebb)

    #Body i/p
    body_ip = Input(name='body_ip',shape=[max_len_b])
    body_ebb = Embedding(vocab_len_b+1, 40, input_length = max_len_b, mask_zero = True, name = 'body_ebb')(body_ip)
    GRU_out_b = GRU(20)(body_ebb)

    #combined with gru o/p
    com = concatenate([GRU_out_t,GRU_out_b])

    #now combined data is being fed to dense layers
    dense1 = Dense(20, activation ='relu')(com)
    dp1 = Dropout(0.5)(dense1)
    bn = BatchNormalization()(dp1)
    dense2 = Dense(20, activation = 'relu')(bn)
    main_op =Dense(10, activation = 'sigmoid', name ='main_op')(dense2)

    model = Model(inputs = [title_ip, body_ip], outputs = [main_op])
    return model

In [ ]:
model = RNN()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 title_ip (InputLayer)       [(None, 18)]                 0         []                            
                                                                                                  
 body_ip (InputLayer)        [(None, 578)]                0         []                            
                                                                                                  
 title_ebb (Embedding)       (None, 18, 100)              6897600   ['title_ip[0][0]']            
                                                                                                  
 body_ebb (Embedding)        (None, 578, 40)              5181768   ['body_ip[0][0]']             
                                                          0                                   

In [ ]:
model.compile(optimizer = 'adam', loss = {'main_op':'categorical_crossentropy'},
              metrics = ['accuracy'])

In [ ]:
results = model.fit({'title_ip':seq_mat_train_t, 'body_ip': seq_mat_train_b}, {'main_op':y_train},
                   validation_data =[{'title_ip':seq_mat_test_t, 'body_ip': seq_mat_test_b}, {'main_op':y_test}],
                   epochs = 3, batch_size = 400)

Epoch 1/3
1032/1032 [==============================] - 2135s 2s/step - loss: 3.7599 - accuracy: 0.2700 - val_loss: 4.5816 - val_accuracy: 0.1685
Epoch 2/3
1032/1032 [==============================] - 2124s 2s/step - loss: 5.9649 - accuracy: 0.1580 - val_loss: 6.5492 - val_accuracy: 0.1136
Epoch 3/3
1032/1032 [==============================] - 2120s 2s/step - loss: 8.5672 - accuracy: 0.1347 - val_loss: 7.5199 - val_accuracy: 0.1191


In [ ]:
(predict_main) = model.predict({'title_ip':seq_mat_test_t, 'body_ip':seq_mat_test_b}, verbose = 1)

4297/4297 [==============================] - 262s 60ms/step


In [ ]:
from sklearn.metrics import classification_report, f1_score

In [ ]:
print(f1_score(y_test,predict_main> 0.40, average = 'samples'))

0.2065231156722066


In [ ]:
print(classification_report(y_test,predict_main>.40))

              precision    recall  f1-score   support

           0       0.12      1.00      0.22     17099
           1       0.15      1.00      0.26     20569
           2       0.07      1.00      0.13      9540
           3       0.08      1.00      0.15     11199
           4       0.07      1.00      0.12      8975
           5       0.16      1.00      0.28     22522
           6       0.17      1.00      0.29     22976
           7       0.12      1.00      0.21     16246
           8       0.14      1.00      0.25     19810
           9       0.08      1.00      0.15     11545

   micro avg       0.12      1.00      0.21    160481
   macro avg       0.12      1.00      0.21    160481
weighted avg       0.13      1.00      0.23    160481
 samples avg       0.12      1.00      0.21    160481



In [ ]:
test.iloc[24]

Id                                                 3693140
Title    Does refreshing mysql query by jQuery ajax cra...
Body     I'm using jQuery .ajax function to load php pa...
Tags                                         [php, jquery]
Name: 38796, dtype: object

In [ ]:
predict_main[24].round(decimals = 2)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)